<a href="https://colab.research.google.com/github/Silvia-Ding/Machine-Learning-Practice-2024/blob/main/Task%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

基本面数据
* 给定一段时间的股票利润表数据，求个股净利润率TTM，需要注意给定的数据是个股每个季度披露的财务数据，计算的净利润率不能用到未来数据，最终输出需要横轴为secu_code，纵轴为tradingdate的净利润matrix项



In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 读取数据
file_path = '/content/QFinancial.csv'
data = pd.read_csv(file_path)

# 确保日期字段为日期格式，并按时间排序
data['InfoPublDate'] = pd.to_datetime(data['InfoPublDate'])
data = data.sort_values(by=['SecuCode', 'InfoPublDate'])

# 用平均值填充缺失值
data['NetProfit'] = data.groupby('SecuCode')['NetProfit'].transform(lambda x: x.fillna(x.mean()))
data['TotalOperatingRevenue'] = data.groupby('SecuCode')['TotalOperatingRevenue'].transform(lambda x: x.fillna(x.mean()))

# 定义 TTM 净利润率计算函数
def calculate_ttm(df):
  df['Net_Profit_TTM'] = df['NetProfit'].rolling(window=4, min_periods=1).sum()
  df['Revenue_TTM'] = df['TotalOperatingRevenue'].rolling(window=4, min_periods=1).sum()
  df['Net_Profit_TTM_Rate'] = df['Net_Profit_TTM'] / df['Revenue_TTM']
  df = df.iloc[4:]  # 跳过前 4 条数据
  return df

# 按证券代码分组并计算 TTM
ttm_data = data.groupby('SecuCode').apply(calculate_ttm)

# 如果 SecuCode 已经是列，先移除它
if 'SecuCode' in ttm_data.columns:
    ttm_data = ttm_data.drop(columns=['SecuCode'])

# 重置索引
ttm_data = ttm_data.reset_index()

# 删除重复值，只保留第一条记录
ttm_data = ttm_data.drop_duplicates(subset=['InfoPublDate', 'SecuCode'], keep='first')

# 构建矩阵
net_profit_matrix = ttm_data.pivot(index='InfoPublDate', columns='SecuCode', values='Net_Profit_TTM_Rate')

# 填充矩阵中的 NaN
net_profit_matrix = net_profit_matrix.fillna(method='ffill').fillna(0)

# 显示矩阵
net_profit_matrix.head(200)


<ipython-input-18-4db38e2e4d10>:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ttm_data = data.groupby('SecuCode').apply(calculate_ttm)
<ipython-input-18-4db38e2e4d10>:42: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  net_profit_matrix = net_profit_matrix.fillna(method='ffill').fillna(0)


SecuCode,1,2,4,5,6,7,8,9,10,11,...,688701,688711,688733,688737,688739,688789,688798,688799,688800,688819
InfoPublDate,,,,,,,,,,,,,,,,,,,,,
2017-04-28,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-05-04,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-05-05,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-05-27,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-07-20,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-03-27,0.226023,0.141477,0.005063,0.04207,0.344888,-0.254814,0.240506,0.043965,-1.220046,0.165096,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-03-28,0.226023,0.141477,0.005063,0.04207,0.344888,-0.254814,0.240506,0.043965,-1.220046,0.165096,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-03-29,0.226023,0.141477,0.005063,0.04207,0.344888,-0.254814,0.240506,0.043965,-1.220046,0.165096,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
